In [0]:
!pip install tensornetwork

     |████████████████████████████████| 122kB 2.9MB/s 
     |████████████████████████████████| 71kB 22.6MB/s 
  Created wheel for opt-einsum: filename=opt_einsum-3.0.0-cp36-none-any.whl size=58490 sha256=1b0860e8b2135b2f0f84c5e933a1b885e215d5db61c3ae89bc7e405b16bc1a8c
  Stored in directory: /root/.cache/pip/wheels/a1/82/aa/e6d68e36a66706a69f890a6072793d2b87730572499c626ed8
Successfully built opt-einsum
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
import numpy as np
import tensorflow as tf
import tensornetwork
from tensornetwork import ncon
net = tensornetwork.TensorNetwork()
sess = tf.InteractiveSession()

# Redes tensoriales en forma de arbol

*    Representados como grafos aciclicos

<img src="https://static.wixstatic.com/media/d91e93_e3a15c2f6d0a45b5913795fe9bbf4384~mv2.png/v1/fill/w_854,h_253,al_c,q_90/Fig18.webp" width="600">

*    **izquierda** Ejemplo red tensorial en forma de arbol (A se considera la raiz de la red)

*    **derecha**  Ramas de la red tensorial

# Invariancia Gauge

Es introducir arbitrariamente la identidad sobre los indices internos de una red tensorial tal que el resultado de la contraccion de la red es el mismo para las infinitas elecciones de los tensores dentro de ella.

<br> **Ejemplo:**

*    La figura de abajo, muestra que se puede introducir la identidad ($I = XX^{-1}$) para cualquier indice interno de la red y no cambia el resultado final de la contraccion de la red.

<img src="https://static.wixstatic.com/media/d91e93_f554522f97944c57966dda0517d5e7cc~mv2.png/v1/fill/w_390,h_403,al_c,q_90/Fig19.webp" width="200">

*   Sin embargo, al unirse una de estas matrices de la identidad con un tensor adjunto cambia su contenido (pero la geometria de la red no cambia)

<img src="https://static.wixstatic.com/media/d91e93_3dd3012583b6444889acec613ce9643a~mv2.png/v1/fill/w_219,h_403,al_c,q_90/Fig2.webp" width="130">


**Nota:** Si bien en algunos aspectos la invariancia gauge es una molestia (ya que implica que las descomposiciones tensoriales nunca son únicas), también puede explotarse para simplificar muchos tipos de operaciones en redes tensoriales. De hecho, la mayoría de los algoritmos requieren la fijación del gauge de manera prescrita para funcionar correctamente. Ahora discutimos varias formas de fijar el grado de libertad del gauge de tal manera que se cree un centro de ortogonalidad y la utilidad de hacerlo.

# Creando un centro de ortogonalidad

Sea $T:\{A,B,C,...\}$ una red tensorial, entonces el tensor $A$ es un centro de ortogonalidad si, para cada rama unida a $A$, las ramas forman una isometria entre sus indices abiertos y los indices conectados a $A$.

<img src="https://static.wixstatic.com/media/d91e93_1475ca0bcc444fadb2076639c6ff8087~mv2.png/v1/fill/w_283,h_240,al_c,q_90/Fig3.webp" width="130">

<img src="https://static.wixstatic.com/media/d91e93_c9eafe1719b24fb6bfbb1db45ea9f06f~mv2.png/v1/fill/w_484,h_338,al_c,q_90/Fig4.webp" width="170">

## Metodos para establecer centro de ortogonalidad

Se describirá el metodo a partir de descomposicion QR (es mas rapido que el SVD).

1.   **Pulling Through**
<br>Si transformamos cada tensor de una rama a uno isometrico (orientado apropiadamente), entonces toda la rama se convierte en un tensor isometrico.    
    - Se inicia orientando cada indice de la red con una flecha que apunte al tensor central $A$
    <br>
    <img src="https://static.wixstatic.com/media/d91e93_4a22ebe9360e452589ed8f19cf88ecdb~mv2.png/v1/fill/w_295,h_250,al_c,q_90/Fig5.webp" width="130">
    - Luego, desde el tensor mas bajo de la rama, realice una descomposición QR (bajo la partición entre flechas entrantes y salientes). A continuación, remodele el tensor "Q" (ortogonal) y absorba la matriz "R" con el tensor conectado a la flecha saliente.
    <br>
    <img src="https://static.wixstatic.com/media/d91e93_2d17c09b5cdd479799f8e2621b4f07ca~mv2.png/v1/fill/w_359,h_148,al_c,q_90/Fig6.webp" width="170">
    <br>
    <img src="https://static.wixstatic.com/media/d91e93_c82ebf2c11824b159a03816bb19004be~mv2.png/v1/fill/w_339,h_160,al_c,q_90/Fig7.webp" width="170">
    -Repita el procedimiento hasta que todos los tensores sean isometricos con respecto a las flechas entrantes y salientes.

<img src="https://static.wixstatic.com/media/d91e93_4ff866b9c05b44ffb14a24a33d1cd2bb~mv2.png/v1/fill/w_950,h_263,al_c,q_90/Fig8.webp" width="400">

In [0]:
##### Creando un centro de ortogonalidad por 'pulling through' 
# definir tensores
d = 3
A = np.random.rand(d,d,d,d); B = np.random.rand(d,d,d) 
C = np.random.rand(d,d,d); D = np.random.rand(d,d,d) 
E = np.random.rand(d,d,d); F = np.random.rand(d,d,d) 
G = np.random.rand(d,d,d)
# iterar descomposiciones QR
DQ, DR = np.linalg.qr(D.reshape(d**2,d)); DQ = DQ.reshape(d,d,d)
EQ, ER = np.linalg.qr(E.reshape(d**2,d)); EQ = EQ.reshape(d,d,d)
Btilda = ncon([B,DR,ER],[[1,2,-3],[-1,1],[-2,2]])
BQ, BR = np.linalg.qr(Btilda.eval().reshape(d**2,d)); BQ = BQ.reshape(d,d,d)
FQ, FR = np.linalg.qr(F.reshape(d**2,d)); FQ = FQ.reshape(d,d,d)
GQ, GR = np.linalg.qr(G.reshape(d**2,d)); GQ = GQ.reshape(d,d,d)
Ctilda = ncon([C,GR],[[1,-2,-3],[-1,1]])
CQ, CR = np.linalg.qr(Ctilda.eval().reshape(d**2,d)); CQ = CQ.reshape(d,d,d)
Aprime = ncon([A,BR,FR,CR],[[1,-2,2,3],[-1,1],[-3,2],[-4,3]])
# Nueva red es formada con tensores: {Aprime,BQ,CQ,DQ,EQ,FQ,GQ}.

# Validar resultado
connectlist = [[3,-5,4,5],[1,2,3],[6,-10,5],[-1,-2,1],[-3,-4,2],[-6,-7,4],[-8,-9,6]]
H0 = ncon([A,B,C,D,E,F,G],connectlist)
H1 = ncon([Aprime,BQ,CQ,DQ,EQ,FQ,GQ],connectlist)
print("dH = ",np.linalg.norm(H0.eval()-H1.eval()) / np.linalg.norm(H0.eval()))

dH =  6.007272352789702e-16




2.   **Ortogonalizacion directa**
<br> Se describirá el metodo a partir de descomposicion espectral para cada rama
    - Calcule la matriz de densidad $\rho$ definida positivamente asociado a cada indice del centro escogido $A$.
    <br>
    <img src="https://static.wixstatic.com/media/d91e93_ff95c759c4fb4b868a401e95c569720f~mv2.png/v1/fill/w_698,h_328,al_c,q_90/Fig26.webp" width="300">
    - Luego, calcule la raiz cuadrada principal $X$ de cada $\rho$
    <br>
    <img src="https://static.wixstatic.com/media/d91e93_5a7d041deda24615ac2a7d367fc85668~mv2.png/v1/fill/w_200,h_135,al_c,q_90/Fig12.webp" width="100">
    - Finalmente, hacemos un cambio de gauge en cada indice del tensor $A$ usando el apropiado $X$ y su correspondiente inversa $X^{-1}$

<img src="https://static.wixstatic.com/media/d91e93_fc0b2b8919454dcfba1c86bbffc7c2e9~mv2.png/v1/fill/w_943,h_268,al_c,q_90/Fig21.webp" width="400">




In [0]:
##### Crear centro de ortogonalidad con 'ortogonalizacion directa' 
# Definir tensores
d = 3
A = np.random.rand(d,d,d,d); B = np.random.rand(d,d,d) 
C = np.random.rand(d,d,d); D = np.random.rand(d,d,d) 
E = np.random.rand(d,d,d); F = np.random.rand(d,d,d) 
G = np.random.rand(d,d,d)
# Calcular matrices de densidad y sus respectivas raices cuadradas principales
rho1 = ncon([B,D,E,B,D,E],[[5,6,-2],[1,2,5],[3,4,6],[7,8,-1],[1,2,7],[3,4,8]])
rho2 = ncon([F,F],[[1,2,-2],[1,2,-1]])
rho3 = ncon([C,G,C,G],[[3,5,-2],[1,2,3],[4,5,-1],[1,2,4]])
d1, u1 = np.linalg.eigh(rho1.eval()); sq_d1 = np.sqrt(abs(d1))
d2, u2 = np.linalg.eigh(rho2.eval()); sq_d2 = np.sqrt(abs(d2))
d3, u3 = np.linalg.eigh(rho3.eval()); sq_d3 = np.sqrt(abs(d3))
X1 = u1 @ np.diag(sq_d1) @ u1.T; X1inv = u1 @ np.diag(1/sq_d1) @ u1.T
X2 = u2 @ np.diag(sq_d2) @ u2.T; X2inv = u2 @ np.diag(1/sq_d2) @ u2.T
X3 = u3 @ np.diag(sq_d3) @ u3.T; X3inv = u3 @ np.diag(1/sq_d3) @ u3.T
# ejecutar cambio gauge
Aprime = ncon([A,X1,X2,X3],[[1,-2,2,3],[-1,1],[-3,2],[-4,3]])
Bprime = ncon([B,X1inv],[[-1,-2,1],[1,-3]])
Fprime = ncon([F,X2inv],[[-1,-2,1],[1,-3]])
Cprime = ncon([C,X3inv],[[-1,-2,1],[1,-3]])
# nueva red es formada por tensores: {Aprime,Bprime,Cprime,D,E,Fprime,G}

# validar resultados
connectlist = [[3,-5,4,5],[1,2,3],[6,-10,5],[-1,-2,1],[-3,-4,2],[-6,-7,4],[-8,-9,6]]
H0 = ncon([A,B,C,D,E,F,G],connectlist)
H1 = ncon([Aprime,Bprime,Cprime,D,E,Fprime,G],connectlist)
print("dH = ",np.linalg.norm(H0.eval() - H1.eval()) / np.linalg.norm(H0.eval()))

dH =  4.441085315907833e-15


**Observacion:**

*   En la práctica, la 'ortogonalización directa' suele ser un cálculo más barato y más fácil de ejecutar.

*   'pulling through' puede ser ventajoso si se desea una alta precisión

# Descomposicion tensorial dentro de redes

Describiremos cómo, creando un centro de ortogonalidad, se puede descomponer optimamente un tensor dentro de una red tal que minimice el error global de toda la red.

<br>
Consideremos una red $\{A,B,C,D,E,F,G\}$ que evaluada da el tensor $H$, entonces bajo reemplazo de $A$ con algún nuevo tensor $A'$, llamamos el nuevo resultado $H'$

<br>
<img src="https://static.wixstatic.com/media/d91e93_8af50d18e9144b9e911090f3ec09c092~mv2.png/v1/fill/w_778,h_438,al_c,q_90/Fig22.webp" width="400">

<b>Teorema</b>

Si $A$ es un centro de ortogonalidad, la diferencia local entre los tensores $||A-A'||$ es igual a la diferencia global entre las redes $||H-H'||$

<img src="https://static.wixstatic.com/media/d91e93_74bd110728044af49946d8a15b6acece~mv2.png/v1/fill/w_750,h_400,al_c,q_90/Fig25.webp" width="400">

<b>Corolario</b> 

Si el tensor de centro de ortogonalidad $A$ es reemplazado por un producto de tensores $A'=A_L.A_R$, entoces la aproximacion optima de rango restringido (la que minimice $||A-A'||$) es tambien optima para minimizar la diferencia global $||H-H'||$

<img src="https://static.wixstatic.com/media/d91e93_c619888421d142d3a3b1d6bba195aaba~mv2.png/v1/fill/w_843,h_518,al_c,q_90/Fig24.webp" width="400">

<b>Observacion:</b> Esto es un resultado significativo ya que una tarea importante en los algoritmos de redes tensoriales (como el DMRG) consiste en descomponer un tensor dentro de la red de tal manera que minimice el error global. 